<a href="https://colab.research.google.com/github/deepartist/DeepFaceLab-Tools/blob/main/MODNet_DeepFaceLab_deepartist.ai_AT_gmail.com.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MODNet  DeepFaceLab colab by Deepartist**
Sources: 

* [Paper](https://arxiv.org/pdf/2011.11961.pdf)
* [Code](https://github.com/ZHKKKe/MODNet)
* [Colab](https://colab.research.google.com/drive/1Pt3KDSc2q7WxFvekCnCLD8P0gBEbxm6J?usp=sharing#scrollTo=he2gSkd1_Hab)

Based on the upscaling Notebook of [Tendeepfake](mailto:tendeepfake@gmail.com
)
## *There is no need for colab pro to use it.*

[Contact me: deepartist](mailto:deepartist.ai@gmail.com)



---



# Install

In [ ]:
#@title Check if you have loaded GPU
gpu = !nvidia-smi -L
gpu= gpu[0]
gpu_mem = !nvidia-smi --query-gpu=memory.total --format=csv
gpu_mem = gpu_mem[1]
gpu_mem_free = !nvidia-smi --query-gpu=memory.free --format=csv
gpu_mem_free = gpu_mem_free[1]
cores = !grep -c ^processor /proc/cpuinfo
cores = cores[0]
mhz = !grep "^[c]pu MHz" /proc/cpuinfo
mhz = mhz[0].split(":")[-1]
!echo "$gpu"
!echo Total Memory: "$gpu_mem" Free: "$gpu_mem_free"
!echo CPU: "$cores" cores @"$mhz" MHz

**Mount your Google drive with auth key**

1. Connect to your google drive to be able to import and export files to/from google drive
2. Click on Link below
3. Login and confirm
4. Ctrl+V auth key and hit return key

**OR mount your Google drive permantly**

1. Create new notebook
2. Change runtime to GPU
3. Click on files, and mount your google drive permantly 
4. Click in first cell
5. Select all cells with shift+down
6. Copy all cells from this notebook
7. No more auth token coyping

**OR SIMPLY SKIP** if you dont want import and export files to/from google drive

In [ ]:
#@title Connect to your gdrive OR permantly mount it

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Clone DFL Git and install needed dependencies
!git clone https://github.com/iperov/DeepFaceLab.git /content/DeepFaceLab
!pip install colorama
print("\nDone")

In [ ]:
#@title Install all dependencies and download weights from org repo

import os
%cd /content
if not os.path.exists('MODNet'):
  !git clone https://github.com/ZHKKKe/MODNet
%cd /content/MODNet/
#!pip install -r demo/video_matting/custom/requirements.txt

pretrained_ckpt = 'pretrained/modnet_photographic_portrait_matting.ckpt'
if not os.path.exists(pretrained_ckpt):
  !wget https://github.com/deepartist/DeepFaceLab-Tools/releases/download/v1.0.3/modnet_photographic_portrait_matting.ckpt 
  !mkdir pretrained
  !mv modnet_photographic_portrait_matting.ckpt pretrained/modnet_photographic_portrait_matting.ckpt

#!python -c "import sys; sys.path.append('/content/MODNet/src'); sys.path.append('/content/MODNet')"


In [ ]:
#@title create custom matting function

def do_matting(indir='/content/inputs/uploads', outdir='/content/no_bg', color=(0,171,57), jpg_output=False):
  import glob
  from pathlib import Path
  import os 
  import cv2


  %cd /content/MODNet

  !python -m demo.image_matting.colab.inference \
      --input-path $indir \
      --output-path '/content/matte' \
      --ckpt-path ./pretrained/modnet_photographic_portrait_matting.ckpt

  from PIL import Image
  import numpy as np
  from tqdm import tqdm
  image_names = sorted(os.listdir(indir))
  for image_name in tqdm(image_names):
    basename = Path(image_name).stem
    matte_name = basename+ '.png'
    image = Image.open(os.path.join(indir, image_name))
    matte = Image.open(os.path.join('/content/matte', matte_name))
    no_bg_img_path = os.path.join(outdir, image_name)

    # obtain predicted foreground
    image = np.asarray(image)
    if len(image.shape) == 2:
      image = image[:, :, None]
    if image.shape[2] == 1:
      image = np.repeat(image, 3, axis=2)
    elif image.shape[2] == 4:
      image = image[:, :, 0:3]
    matte = np.repeat(np.asarray(matte)[:, :, None], 3, axis=2) / 255
    
    foreground = image * matte + np.full(image.shape, color) * (1 - matte)
    foreground_img = Image.fromarray(np.uint8(foreground))
    foreground_img.save(no_bg_img_path)
    
      

# Matting Process

In [ ]:
#@title 0. Delete old example and result files
!rm -rf cd /content/inputs/uploads
!mkdir -p /content/inputs/uploads
!rm -rf /content/matte
!mkdir -p /content/matte
!rm -rf  /content/no_bg/
!mkdir -p  /content/no_bg/

!rm -rf /content/matte.zip
!rm -rf /content/dfl_images.zip

print("\nDone!")

## Upload files
You can upload single or multiple files or your zip aligend folder or faceset.pak

### files and zipped aligned folder

In [ ]:
#@title 1a. Upload one or more (DFL).jpg files to mat *OR* 
%cd /content/inputs/uploads
from google.colab import files
uploaded = files.upload()
%cd /content/
print("\nDone!")

In [ ]:
#@title 1b. Upload aligned.zip and extract to imgs *OR*
%cd /content/
!rm aligned.zip
from google.colab import files
uploaded = files.upload()
%cd /content/
!7z e '/content/aligned.zip' -o'/content/inputs/uploads'
print("\nDone!")

In [ ]:
#@title 1c. copy aligned.zip from your drive and extract to imgs
import os

file_on_drive = "/content/drive/MyDrive/aligned.zip" #@param {type:"string"}
%cd /content/
!rm aligned.zip
!cp $file_on_drive /content/
!7z e '/content/aligned.zip' -o'//content/inputs/uploads'
print("\nDone!")

### faceset.pak

In [ ]:
#@title 2b. Upload faceset.pak and extract to imgs *OR*
%cd /content/inputs/uploads
!rm faceset.pak
from google.colab import files
uploaded = files.upload()
%cd /content
cmd = "DeepFaceLab/main.py util --input-dir /content/inputs/uploads --unpack-faceset"
!python $cmd
print(f"Unpacking done!")
print("\nDone!")

In [ ]:
#@title 2d. copy faceset.pak from your drive and extract to imgs
import os

file_on_drive = "/content/drive/MyDrive/faceset.pak" #@param {type:"string"}
%cd /content/
!rm faceset.pak
!cp $file_on_drive /content/inputs/uploads
cmd = "DeepFaceLab/main.py util --input-dir /content/inputs/uploads --unpack-faceset"
!python $cmd
print(f"Unpacking done!")
print("\nDone!")

## Matting 

In [ ]:
#@title 3. Optional Resize if original faceset if too small (<256) Matting works best with 512x512
%cd /content/
cmd = "DeepFaceLab/main.py facesettool resize --input-dir /content/inputs/uploads"
!python $cmd

In [ ]:
#@title 4. Save DFL metadata and start matting and restore
background_color = "green" #@param ["green", "red", "blue", "white", "black"]
color_dict ={'green': (0,171,57), 'blue': (46,0,139), 'red': (234, 29, 40), 'black': (0,0,0), 'white':(255,255,255)}

#@markdown Use custom color code
custom_color_code = False #@param {type:"boolean"}
custom_color_code_red = 46 #@param {type:"slider", min:0, max:255, step:1}
custom_color_code_green = 113 #@param {type:"slider", min:0, max:255, step:1}
custom_color_code_blue = 209 #@param {type:"slider", min:0, max:255, step:1}
if custom_color_code:
  color = (custom_color_code_red, custom_color_code_green, custom_color_code_blue)
else:
  color = color_dict[background_color]

#@markdown Ignore errors if you dont use DFLIMG
%cd /content/
import glob
from pathlib import Path
cmd = "DeepFaceLab/main.py util --input-dir /content/inputs/uploads --save-faceset-metadata"
!python $cmd
ext = glob.glob("/content/inputs/uploads/*")[0].split(".")[-1]
if ext.lower() is not "jpg":
  jpg_output=False
  print("Keeping original file format")
else:
  jpg_output=True
!mv /content/inputs/uploads/meta.dat /content/no_bg
print("Start matting")
do_matting(color=color, jpg_output=jpg_output)
print("Matting done")
%cd /content/

cmd = "DeepFaceLab/main.py util --input-dir /content/no_bg --restore-faceset-metadata"
!python $cmd
print("\nDone!")

## Save your files

###Images with removed Backgrounds

#### Zips

In [ ]:
#@title 5a. Zip images faces
%rm /content/dfl_images.zip
%rm /content/no_bg/faceset.pak
!7z a -tzip /content/dfl_images.zip -w /content/no_bg/*
print("\nDone!")

In [ ]:
#@title 5b. Download dfl_enhanced.zip *OR*
from google.colab import files
files.download("/content/dfl_enhanced.zip")
print("\nDone!")

In [ ]:
#@title 5c. Copy dfl_enhanced.zip to gdrive
from google.colab import files
folder_on_drive = "/content/drive/MyDrive/" #@param {type:"string"}

!cp -f /content/dfl_images.zip $folder_on_drive
print("\nDone!")

#### Faceset.pak

In [ ]:
#@title 6a. Repack to faceset.pak 
%cd /content
import glob 
files = glob.glob("/content/no_bg/*jpg")
if len(files) > 0:
  %rm /content/no_bg/faceset.pak
  cmd = "DeepFaceLab/main.py util --input-dir /content/no_bg --pack-faceset"
  !python $cmd
  !cp /content/no_bg/faceset.pak /content/faceset.pak
else: 
  print('No imgs found')


In [ ]:
#@title 6b. Download faceset.pak *OR*
from google.colab import files
files.download("/content/no_bg/faceset.pak")
print("\nDone!")

In [ ]:
#@title 6c. Copy faceset.pak to gdrive
from google.colab import files
folder_on_drive = "/content/drive/MyDrive/" #@param {type:"string"}
!cp -f /content/no_bg/faceset.pak $folder_on_drive
print("\nDone!")

### Matte

In [ ]:
#@title 7a. Zip mattte files 
%rm /content/matte.zip
!7z a -tzip /content/matte.zip -w /content/matte/*
print("\nDone!")

In [ ]:
#@title 7b. Download matte.zip *OR*
from google.colab import files
files.download("/content/matte.zip")
print("\nDone!")

In [ ]:
#@title 7c. Copy matte.zip to gdrive
from google.colab import files
folder_on_drive = "/content/drive/MyDrive/" #@param {type:"string"}
!cp -f /content/matte.zip $folder_on_drive
print("\nDone!")